In [10]:
import pandas as pd
import numpy as np

In [11]:
# 1. Load all datasets
rawg = pd.read_csv("data/rawg_balanced_2000_2025.csv")
steam = pd.read_csv("data/steam.csv")
sales = pd.read_csv("data/vgsales.csv")

print("RAWG:", rawg.shape)
print("Steam:", steam.shape)
print("Sales:", sales.shape)

RAWG: (4000, 6)
Steam: (27075, 18)
Sales: (16598, 11)


In [12]:
# 2. Standardize column names
rawg.rename(columns={'genres': 'genre'}, inplace=True)
steam.rename(columns={'genres': 'genre'}, inplace=True)
sales.rename(columns={'Genre': 'genre', 'Name': 'name', 'Platform': 'platform',
                      'Year': 'year', 'Global_Sales': 'global_sales'}, inplace=True)
for df in [rawg, steam, sales]:
    if 'genre' in df.columns:
        df['genre'] = df['genre'].astype(str).str.lower().str.strip()

In [13]:
# 3. Remove unrated or zero-rated games from RAWG
rawg = rawg[rawg['rating'].notna() & (rawg['rating'] > 0)]
print("\nAfter removing unrated or zero-rating games:", rawg.shape)


After removing unrated or zero-rating games: (1480, 6)


In [14]:
# 4. Keep only gaming-related entries
if 'genre' not in rawg.columns:
    rawg['genre'] = np.nan
game_keywords = [
    "action", "adventure", "rpg", "casual", "indie", "simulation", "strategy",
    "sports", "racing", "puzzle", "horror", "shooter", "platformer",
    "fighting", "survival", "arcade", "visual novel"
]

rawg = rawg[rawg['genre'].str.contains('|'.join(game_keywords), case=False, na=False)]
steam = steam[steam['genre'].str.contains('|'.join(game_keywords), case=False, na=False)]
sales = sales[sales['genre'].str.contains('|'.join(game_keywords), case=False, na=False)]

print("\nAfter keeping only gaming-related rows:")
print("RAWG:", rawg.shape)
print("Steam:", steam.shape)
print("Sales:", sales.shape)


After keeping only gaming-related rows:
RAWG: (1448, 6)
Steam: (26822, 18)
Sales: (12485, 11)


In [15]:
# 5. Remove adult/non-gaming content
exclude_keywords = [
    "sexual", "nudity", "porn", "adult", "hentai",
    "violence", "nsfw", "mature", "gore",
    "audio production", "video production", "utilities",
    "education", "software", "design", "animation"
]

rawg = rawg[~rawg['genre'].str.contains('|'.join(exclude_keywords), case=False, na=False)]
steam = steam[~steam['genre'].str.contains('|'.join(exclude_keywords), case=False, na=False)]
sales = sales[~sales['genre'].str.contains('|'.join(exclude_keywords), case=False, na=False)]

print("\nAfter removing adult/non-gaming genres:")
print("RAWG:", rawg.shape)
print("Steam:", steam.shape)
print("Sales:", sales.shape)


After removing adult/non-gaming genres:
RAWG: (1443, 6)
Steam: (25985, 18)
Sales: (12485, 11)


In [16]:
# 6. Clean columns and handle years
if 'released' in rawg.columns:
    rawg['year'] = pd.to_datetime(rawg['released'], errors='coerce').dt.year

sales['year'] = pd.to_numeric(sales['year'], errors='coerce')
rawg.dropna(subset=['year'], inplace=True)
sales.dropna(subset=['year'], inplace=True)

In [17]:
# 7. Save Clean Datasets
rawg.to_csv("data/rawg_clean.csv", index=False)
steam.to_csv("data/steam_clean.csv", index=False)
sales.to_csv("data/vgsales_clean.csv", index=False)

print("\nClean datasets saved:")
print("- data/rawg_clean.csv")
print("- data/steam_clean.csv")
print("- data/vgsales_clean.csv")


Clean datasets saved:
- data/rawg_clean.csv
- data/steam_clean.csv
- data/vgsales_clean.csv
